### Madden Ultimate Team (MUT) Auction House Analyzer
#### Matthew Johnson, March 21, 2018

A set in madden is where you buy n specific cards to complete the set and recieve a more valuable player in return.

Currently:
- Currently running two parallel arrays of set players and their ids
- Keep track of target player and target_id 
- Use selenium to open webdriver Firefox
- Manually log into 'Muthead' (Must be logged in to refresh auction prices)
- Selenium refreshes all of the auction prices of relevant players page by page
- Use beautifulsoup to scrape from three tables: Current Auctions, Completed Sales, and 'Stats for Nerds'
- Using the minimum BuyNow price for each player, the price to complete the set is calculated
- Using this, an average of last X sales of player and taking into account the 10% tax fee on the Auction House a conditional is triggered to pursue the set based on expected profit

Current Pitfalls:
- Need to manually login
- Poor flow
- Likely needs proper use of .sleep() 

Planned:
- store data, pickle?
- to also use for non-set (rare) players


In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import urllib
import time

In [2]:
# RUN THIS

browser = webdriver.Firefox()

URL = 'https://www.muthead.com/twitch-login?returnUrl=%2f'

browser.get(URL)

In [3]:
#manually log in

# username = 'XXXXXXXX'
# password = 'XXXXXXXX'

# data frame to store scraped data
data = pd.DataFrame(columns=['Name', 'Min Price', 'Average', 'Date'], index=[0])

## Methods<br>
#### All take in a beautiful soup object:
- retrieve_prices(soup): returns a dataframe of current auctions
- retrieve_stats(soup): returns a dataframe of stats for the last 100 sales such as quartile prices, min, avg, etc.
- retrieve_sales(soup): returns dataframe of last 10 completed sales

In [4]:
def retrieve_prices(soup):    # method that returns a dataframe of current auctions
    
    print (soup.title.text) # print player page title
    
    div_auctions = soup.find_all('div', class_="player-prices-live-auctions")
    
    for div in div_auctions:
        div.find('div', class_="player-prices-live-auctions")
    
    auction_table = soup.select_one("table:nth-of-type(1)")
    
    rows = auction_table.find_all('tr')
    num_rows = min ( len(rows), 5 )
    #title row
    row = rows[0]
    td_tags = row.find_all('td')
    th_tags = row.find_all('th') 

    titles = []

    for tag in th_tags:
        titles.append(str(tag)[4:-5]) #remove tags
    
    num_cols = len(titles)

    new_table = pd.DataFrame(columns=titles, index=range(0,num_rows-1))

    for i in range(1,num_rows):
        row = rows[i]

        td_tags = row.find_all('td')

        values = []

        for val in td_tags:
            values.append(str(val)[4:-5])
    
        #time remaining
        time_rem = values[0][58:67].replace("<", "").replace(">", "")
        num_bids = values[1]
        curr_bid = values[2]
        buy_now = values[3]
        nums = [time_rem, num_bids, curr_bid, buy_now]
        j=0
    
        for title in titles:
            new_table[title][i-1] = nums[j]
            j += 1
    
    return new_table

In [5]:
def retrieve_stats(soup):  # method to return dataframe of stats for last 100 sales such as quartile prices, min, avg
    
    div_hist = soup.find_all('div', class_="player-details-right")
    
    for div in div_hist:
        t = div.find('div', class_="player-prices-advanced-statistics")
        hist_table = t.select_one("table:nth-of-type(1)")
    
        rows = hist_table.find_all('tr')

    new_table2 = pd.DataFrame(columns=['Q', 'Price'], index=[0,1,2,3,4,5,6])
    
    for i in range(0,7): #first row of prices
        row = rows[i]
        td_tags = row.find_all('td')
        
        values = []

        for val in td_tags:
            values.append(str(val)[4:-5])
    
        #time remaining
        ca = values[0]
        cb = values[1][14:]
    
        nums = [ca,cb]
        #print(nums) # debug
        new_table2['Q'][i] = ca
        new_table2['Price'][i] = cb
    
    return new_table2

In [6]:
def retrieve_sales(soup):    # method to return dataframe of last n completed sales ( <= 10?)
    
    div_sales = soup.find_all('div', class_="player-prices-completed-sales")

    sales_table = soup.select_one("table:nth-of-type(2)")
    rows = sales_table.find_all('tr')
    num_rows = min ( len(rows), 10 )
    
    row = rows[0] #title row
    td_tags = row.find_all('td')
    th_tags = row.find_all('th') 
    
    titles = []

    for tag in th_tags:
        titles.append(str(tag)[4:-5]) #remove tags
    
    num_cols = len(titles)
    new_table3 = pd.DataFrame(columns=titles, index=range(0,num_rows-1))
   
    for i in range(1,num_rows):
        row = rows[i]

        td_tags = row.find_all('td')

        values = []

        for val in td_tags:
            values.append(str(val)[4:-5])
            
        time_sold = values[0][119:140].replace("<", "").replace(">", "")
        price_sold = values[1].replace(".", "").replace(",", "")
        
        nums = [time_sold, price_sold] #print(nums) # debug
        j=0
    
        for title in titles:
            new_table3[title][i-1] = nums[j]
            j += 1
            
    new_table3['Price'] = pd.to_numeric( new_table3['Price'] )
    return new_table3

# start here
Place players and their respective IDs in parallel arrays.

In [7]:
# PLAYERS WE WANT TO SEARCH ARE DIVIDED INTO TWO LISTS OF NAMES AND PLAYERIDS

# Set Player Names
#player_names = ['Any Dalton', 'Laremy Tunsil', 'Lamer Miller', 'Eric Decker', 'Ben Watson', 'Spencer Long', 'Brian Lafell', 'Don Barclay']
player_names = ['Kirk Cousins', 'Jahri Evans', 'Zach Miller', 'Martavis Bryant', 'Russell Okung', 'Brandon Fusco', \
               'Ryan Mathews', 'Dwayne Harris']
# Set Player IDs
#player_ids = ['34093', '34094', '34095', '34096', '34101', '34102', '34103', '34104']
player_ids = ['34165', '34166', '34167', '34168', '34173', '34174', '34175', '34176'  ]

**To refresh prices:**

In [ ]:
link_start = 'https://www.muthead.com/18/players/prices/'
link_end = '/playstation-4/'
link_end_ref = '/playstation-4/refresh/'
start = time.time()
print("timestart")

for k in range(0, len(player_ids)):
    linky = link_start + player_ids[k] + link_end_ref
    browser.get(linky)
    time.sleep(5)
#browser.quit()  # uncomment to close browser but will log you out of site

end = time.time()
print('Time in seconds for completion:\t', end - start)

**Scraping the data:**

In [9]:
start = time.time() # INITIALIZE TIMER FOR COMPUTING TIME
print("timestart")

link_start = 'https://www.muthead.com/18/players/prices/' # LINK PREFIX/POSTFIX
link_end = '/playstation-4/'

min_prices = [] # LIST FOR LOWEST BUYITNOW PRICE PER PLAYER
N = len(player_names)

for i in range(0,N):
    mut_url = link_start + player_ids[i] + link_end
    r = urllib.request.urlopen(mut_url).read()
    soup = BeautifulSoup(r,"lxml")
    time.sleep(2) 
    print('\n= = = = = Current Prices for', player_names[i], '= = = = = ')
    print('retrieving prices')
    df = retrieve_prices(soup)
    time.sleep(2) 
    print('retrieving stats')
    df2 = retrieve_stats(soup)
    time.sleep(2)
    print()
    
    current_min = df['Buy Now Price'][0].replace(",", "")
    lower_q = df2['Price'][5].replace(",", "")
    
    # potential deal
    if ( float(current_min) < float(lower_q) ):
        print ('======DEAL======')
        
    min_prices.append(current_min)
    
    temp = time.time()
    print('Time elapsed:\t', temp - start)
    
end = time.time()
print('Time in seconds for completion:\t', end - start)

timestart

= = = = = Current Prices for Kirk Cousins = = = = = 
retrieving prices
Kirk Cousins Prices - Madden 18 - 93 OVR NFL Replays - Muthead 
retrieving stats

Time elapsed:	 7.441845178604126

= = = = = Current Prices for Jahri Evans = = = = = 
retrieving prices
Jahri Evans Prices - Madden 18 - 92 OVR NFL Replays - Muthead 
retrieving stats

Time elapsed:	 15.211973905563354

= = = = = Current Prices for Zach Miller = = = = = 
retrieving prices
Zach Miller Prices - Madden 18 - 91 OVR NFL Replays - Muthead 
retrieving stats

======DEAL======
Time elapsed:	 22.66271710395813

= = = = = Current Prices for Martavis Bryant = = = = = 
retrieving prices
Martavis Bryant Prices - Madden 18 - 91 OVR NFL Replays - Muthead 
retrieving stats

Time elapsed:	 30.36227011680603

= = = = = Current Prices for Russell Okung = = = = = 
retrieving prices
Russell Okung Prices - Madden 18 - 89 OVR NFL Replays - Muthead 
retrieving stats

Time elapsed:	 38.11774015426636

= = = = = Current Prices for Bra

In [10]:
min_prices

['135000', '86500', '84500', '64000', '17750', '29000', '7900', '18000']

**Calculate the price we can get for the player obtained by completing the set and determine whether it is a worthwhile venture:**

In [11]:
start = time.time()

summ = 0
for i in range(0,8):
    summ+= int(min_prices[i])
    
print('Cost to Build:\t', summ)

#target = 'Trent Williams'
#target_id = '34091'
target = 'Dez Bryant'
target_id = '34163'

mut_url = link_start + target_id + link_end
r = urllib.request.urlopen(mut_url).read()
soup = BeautifulSoup(r,"lxml")
time.sleep(5) 
print('retrieving prices')
df = retrieve_prices(soup)
time.sleep(3) 
print('retrieving stats')
df2 = retrieve_stats(soup)
print()
df3 = retrieve_sales(soup)

current_min = df['Buy Now Price'][0].replace(",", "")
lower_q = df2['Price'][5].replace(",", "")
last_x_mean = df3['Price'].mean()
       
cur_min = float (current_min )
after_tax = cur_min*0.9

if ( after_tax > summ):
    print('==+++===== BUY! BUY! BUY! =====+++==\n')
else:
    print('X X X X X X NOT WORTH IT! X X X X X X\n')
    
for i in range(0, 8):
    print( player_names[i] + '\t' + min_prices[i] )

print ('\nCost to build:\t\t', summ)
print ('Lowest ' + target +':\t', int(cur_min))
print ('After Tax :\t\t', int(after_tax))
print ('Last x mean:\t\t', int(last_x_mean) )
print('Potential profit:\t', int(after_tax-summ))

end = time.time()
timey = end - start
timey = str(round(timey, 2))

print('\nTime in seconds for completion:\t', timey)

Cost to Build:	 442650
retrieving prices
Dez Bryant Prices - Madden 18 - 97 OVR NFL Replays - Muthead 
retrieving stats

X X X X X X NOT WORTH IT! X X X X X X

Kirk Cousins	135000
Jahri Evans	86500
Zach Miller	84500
Martavis Bryant	64000
Russell Okung	17750
Brandon Fusco	29000
Ryan Mathews	7900
Dwayne Harris	18000

Cost to build:		 442650
Lowest Dez Bryant:	 459000
After Tax :		 413100
Last x mean:		 482166
Potential profit:	 -29550

Time in seconds for completion:	 9.86


###### Matthew Johnson, March 21, 2018